# Polaris Catalog Credential Vending Fine-Grained RBAC per Table Demo 🚀

<div class="alert alert-info">
Walk through how Polaris credential vending with Ceph’s IAM/STS APIs enforces table-level access control in Iceberg.
</div>

## Table of Contents
- [Load Tokens](#load-tokens)
- [Sessions Setup](#sessions-setup)
- [Charlie: Inspect `polaris.prod_ns` Tables](#charlie-inspect)
- [Charlie: Load CSV into `products` Table](#charlie-load)
- [Charlie: Verify `products` Table](#charlie-verify)
- [Alice: List Tables in `prod_ns` (❌)](#alice-list)
- [Alice: Insert into `prod_ns.eu_user`](#alice-insert-eu)
- [Alice: Read from `prod_ns.products`](#alice-read)
- [Alice: Products per Category Chart](#alice-chart)
- [Alice: Insert into `prod_ns.us_user` (❌)](#alice-insert-us)


<a id="load-tokens"></a>
## 1️⃣ Load OAuth2 Tokens

Pull in the JSON tokens Terraform minted for Charlie, Alice and Bob, and display them cleanly with **pprint**.

In [1]:
import json, pathlib, pprint

TOKENS = json.loads(pathlib.Path('/home/jovyan/work/tokens.json').read_text())
CHARLIE_TOKEN = TOKENS['charlie']
ALICE_TOKEN   = TOKENS['alice']
BOB_TOKEN     = TOKENS['bob']

print('Tokens loaded for:')
pprint.pprint(TOKENS, width=1)

<a id="sessions-setup"></a>
## 2️⃣ Spark Sessions Setup

**Charlie** initializes a SparkSession with admin credentials (polaris catalog).  
Then we **clone** that session for Alice and Bob, swapping in their tokens to scope permissions per table.

In [2]:
from pyspark.sql import SparkSession
from py4j.protocol import Py4JJavaError

charlie = (
    SparkSession.builder
        .appName('polaris-admin')
        .config('spark.jars.packages',
            'org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.7.1,org.apache.hadoop:hadoop-aws:3.4.0')
        .config('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions')
        .config('spark.sql.catalog.polaris', 'org.apache.iceberg.spark.SparkCatalog')
        .config('spark.sql.catalog.polaris.type', 'rest')
        .config('spark.sql.catalog.polaris.uri', 'http://polaris:8181/api/catalog')
        .config('spark.sql.catalog.polaris.warehouse', 'prod')
        .config('spark.sql.catalog.polaris.header.X-Iceberg-Access-Delegation', 'vended-credentials')
        .config('spark.sql.catalog.polaris.token', CHARLIE_TOKEN)
        .config('spark.sql.defaultCatalog', 'polaris')
        .getOrCreate()
)
print('Session: Charlie (admin)')

# Clone to Alice & Bob
alice = charlie.newSession()
alice.conf.set('spark.sql.catalog.polaris.token', ALICE_TOKEN)
print('Session: Alice →', ALICE_TOKEN[:8] + '...')

bob = charlie.newSession()
bob.conf.set('spark.sql.catalog.polaris.token', BOB_TOKEN)
print('Session: Bob   →', BOB_TOKEN[:8] + '...')

<a id="charlie-inspect"></a>
## 3️⃣ Charlie: Inspect `polaris.prod_ns` Tables

As **admin**, Charlie lists all tables in the `prod_ns` namespace.

In [3]:
import pandas as pd
from IPython.display import display

tbls = charlie.sql('SHOW TABLES IN polaris.prod_ns').toPandas()
display(tbls)

<a id="charlie-load"></a>
## 4️⃣ Charlie: Load Products CSV into `products` Table

Read the local `products.csv` file and **append** it into the Iceberg table `polaris.prod_ns.products`.

In [4]:
df = (
    charlie.read
        .option('header', True)
        .option('inferSchema', True)
        .csv('/home/jovyan/work/products.csv')
)
df.writeTo('polaris.prod_ns.products').append()
print('✅ Data loaded into products table')

<a id="charlie-verify"></a>
## 5️⃣ Charlie: Verify `products` Table

Confirm rows in `polaris.prod_ns.products` by selecting the first five entries.

In [5]:
pdf = charlie.sql('SELECT * FROM polaris.prod_ns.products LIMIT 5').toPandas()
display(pdf)

<a id="alice-list"></a>
## 6️⃣ Alice: List Tables in `prod_ns` (❌)

<div class="alert alert-danger">
Alice does **not** have the `LIST_TABLES` privilege in `polaris.prod_ns`, so this will fail.
</div>

In [6]:
from IPython.display import HTML, display
import pyspark

print('SHOW TABLES as Alice...')
try:
    alice.sql('SHOW TABLES IN polaris.prod_ns').show()
except (pyspark.sql.utils.AnalysisException, Py4JJavaError) as e:
    msg = e.desc if hasattr(e, 'desc') else e.java_exception.getMessage()
    display(HTML(f"<div class='alert alert-danger'><b>🔒 {msg}</b></div>"))

<a id="alice-insert-eu"></a>
## 7️⃣ Alice: Insert into `eu_user` Table

<div class="alert alert-success">
Alice’s **eu_data_admin** role grants INSERT on `polaris.prod_ns.eu_user`.
</div>

In [7]:
import pandas as pd
from IPython.display import display

print('INSERT into eu_user...')
alice.sql("""
INSERT INTO polaris.prod_ns.eu_user VALUES
  (1001,'alice.w@example','alice@example.com','FR',TIMESTAMP '2025-05-21 10:15:00',NULL)
""")
pdf_eu = alice.sql('SELECT * FROM polaris.prod_ns.eu_user LIMIT 5').toPandas()
display(pdf_eu)

<a id="alice-read"></a>
## 8️⃣ Alice: Read from `products` Table

<div class="alert alert-info">
Alice can **SELECT** from `polaris.prod_ns.products` thanks to her read privileges.
</div>

In [8]:
import pandas as pd
from IPython.display import display

print('SELECT from products...')
pdf3 = alice.sql('SELECT * FROM polaris.prod_ns.products LIMIT 5').toPandas()
display(pdf3)

<a id="alice-chart"></a>
### 📊 Alice: Products per Category Chart for `products`

Visualize row counts per category from `polaris.prod_ns.products`.

In [9]:
prod_df = alice.sql(
    '''
    SELECT category, COUNT(*) AS cnt
    FROM polaris.prod_ns.products
    GROUP BY category
    '''
).toPandas()
prod_df.plot.bar(x='category', y='cnt', title='Products per Category')

<a id="alice-insert-us"></a>
## 9️⃣ Alice: Insert into `us_user` Table (❌)

<div class="alert alert-danger">
Alice’s role does **not** include INSERT on `polaris.prod_ns.us_user`, so this will fail.
</div>

In [10]:
from IPython.display import HTML, display
import pyspark

print('INSERT as Alice...')
try:
    alice.sql("INSERT INTO polaris.prod_ns.us_user VALUES (2001,'x','x','US',CURRENT_TIMESTAMP,NULL)")
except (pyspark.sql.utils.AnalysisException, Py4JJavaError) as e:
    msg = e.desc if hasattr(e, 'desc') else e.java_exception.getMessage()
    display(HTML(f"<div class='alert alert-danger'><b>🔒 {msg}</b></div>"))